https://voice.mozilla.org/en/datasets

### Content
Total hours: ? h.

Language: Spanish.

Number of speakers: ???.

Type of speech: ??? # Clean speech.


#### Collection methodology

????

### Extra
Collected by: 

### License
License : Public Domain

### Downloading 
#### Method 


In [ ]:
import glob
import os
#import subprocess
#import tarfile
#import wget

In [ ]:
#%matplotlib inline
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import IPython.display as ipd
from ipywidgets import interact
import ipywidgets as widgets
import re

In [ ]:
import soundfile as sf
#import IPython.display as ipd
#from ipywidgets import interact
#import ipywidgets as widgets
from pathlib import Path


In [ ]:
%ls

In [ ]:
#%ls asr-spanish-v1-carlfm01\audios\0000df16-47ea-428f-8367-df2ce365d5c4.wav

In [ ]:
#%ls asr-spanish-v1-carlfm01\asr-spanish-v1-carlfm01\audios\0000df16-47ea-428f-8367-df2ce365d5c4.wav

In [ ]:
pwd

In [ ]:
egg_folder_name = os.path.join('/gong-asr/kaldi-12/Domain-specific-ESPnet','espnet','egs',
                        'spanish_common_voice','asr1')

data_folder_name = os.path.join(egg_folder_name, 'raw_data', 'CommonVoiceSpanish', 'decompressed')

kaldi_ready_data_folder_name = os.path.join(egg_folder_name, 'data')

In [ ]:
! ls -l {data_folder_name} | wc -l

In [ ]:
description_file_name = os.path.join(data_folder_name,'train.tsv')

In [ ]:
%ls -l {description_file_name}

In [ ]:
df = pd.read_csv(description_file_name, sep='\\t')

In [ ]:
df.head()

In [ ]:
len(df.index) == len(set(df.index))

In [ ]:
df_raw = df.loc[df.index[0]]
df_raw

In [ ]:
def segment_by_df_raw(df_raw):
    audio, sample_rate = librosa.load(os.path.join(data_folder_name,'clips',df_raw.path))
    plt.rcParams['figure.figsize'] = (15,7)
    plt.title(f'Waveform of Audio Example: {df_raw["path"]}')
    plt.ylabel('Amplitude')

    print(df_raw['sentence'])
    print(f'sample_rate: {sample_rate}')
    _ = librosa.display.waveplot(audio)
    return ipd.Audio(audio, rate=sample_rate)

In [ ]:
segment_by_df_raw(df_raw)

In [ ]:
def segment_by_idx(idx):
    return segment_by_df_raw(df.iloc[idx])

In [ ]:
interact(segment_by_idx, idx=widgets.IntSlider(min=0, max=df.shape[0]-1, step=1, value=0));

In [ ]:
train_test = 'test'
kaldi_ready_md_folder = os.path.join(kaldi_ready_data_folder_name, 'test')
kaldi_ready_audio_folder = os.path.join(kaldi_ready_data_folder_name, 'test')

In [ ]:
! ls -l {kaldi_ready_md_folder}

In [ ]:
! head -n5 {os.path.join(kaldi_ready_md_folder, 'wav.scp')}

In [ ]:
! head -n5 {os.path.join(kaldi_ready_md_folder, 'text')}

In [ ]:
wavscp_df = pd.read_csv(os.path.join(kaldi_ready_md_folder, 'wav.scp'), sep=' ',header=None)\
.set_index(0)
wavscp_df.columns = ['wav']
wavscp_df.head()

In [ ]:
with open(os.path.join(kaldi_ready_md_folder, 'text'), 'r') as text_fd:
    text_str = text_fd.read()
text_df = pd.DataFrame(np.array([(lambda x: (x[0], ' '.join(x[1:])))(ent.split(' ')) for ent in text_str.split('\n') if ''!=ent])).set_index(0)
text_df.columns = ['text']
text_df.head()

In [ ]:
set(text_df.index == wavscp_df.index)

In [ ]:
parsed_df = wavscp_df.join(text_df)
parsed_df.head()

In [ ]:
def segment_by_parsed_df_raw(df_raw):
    audio, sample_rate = sf.read(os.path.join(egg_folder_name, df_raw.wav))
    plt.rcParams['figure.figsize'] = (15,7)
    plt.title(f'Waveform of Audio Example: {df_raw["wav"]}')
    plt.ylabel('Amplitude')

    print(df_raw['text'])
    print(f'sample_rate: {sample_rate}')                            
    _ = librosa.display.waveplot(audio)
    return ipd.Audio(audio, rate=sample_rate)

In [ ]:
def parsed_segment_by_idx(idx):
    return segment_by_parsed_df_raw(parsed_df.iloc[idx])

In [ ]:
! ls -l /gong-asr/kaldi-12/Domain-specific-ESPnet/espnet/egs/spanish_common_voice/asr1/data/train/downloads/comvoice/common_voice_es_19698530.wav

In [ ]:
interact(parsed_segment_by_idx, idx=widgets.IntSlider(min=0, max=parsed_df.shape[0]-1, step=1, value=0));

## json labels

In [ ]:
train_json_local_path = 'dump/train_set/deltafalse/data_unigram5000.json'
train_dev_json_local_path = 'dump/train_dev/deltafalse/data_unigram5000.json'
test_json_local_path = 'dump/test/deltafalse/data_unigram5000.json'

In [ ]:
import json

In [ ]:
train_dict = json.loads(open(os.path.join(egg_folder_name, train_json_local_path),'r').read())
train_dev_dict = json.loads(open(os.path.join(egg_folder_name, train_dev_json_local_path),'r').read())
test_dict = json.loads(open(os.path.join(egg_folder_name, test_json_local_path),'r').read())

In [ ]:
print(len(list(train_dict['utts'].keys())))
print(len(list(train_dev_dict['utts'].keys())))
print(len(list(test_dict['utts'].keys())))

In [ ]:
list(train_dict['utts'].keys())[0]

In [ ]:
train_dict['utts'][list(train_dict['utts'].keys())[0]]['utt2spk'] 

In [ ]:
train_spks = (lambda x: {x['utts'][k]['utt2spk'] for k in x['utts'].keys()})(train_dict)
len(train_spks)

In [ ]:
train_dev_spks = (lambda x: {x['utts'][k]['utt2spk'] for k in x['utts'].keys()})(train_dev_dict)
len(train_dev_spks)

In [ ]:
test_spks = (lambda x: {x['utts'][k]['utt2spk'] for k in x['utts'].keys()})(test_dict)
len(test_spks)

check there is no intersection of speekers

In [ ]:
print(len(np.intersect1d(train_spks, train_dev_spks)))
print(len(np.intersect1d(train_spks, test_spks)))
print(len(np.intersect1d(train_dev_spks, test_spks)))

In [ ]:
train_input_shapes = (lambda x: [x['utts'][k]['input'][0]['shape'] for k in x['utts'].keys()])(train_dict)
train_output_shapes = (lambda x: [x['utts'][k]['output'][0]['shape'] for k in x['utts'].keys()])(train_dict)
print(set(np.array(train_input_shapes).T[1])) # should be correspondent to 80 fbanks 
print(set(np.array(train_output_shapes).T[1])) # should be correspondent to 5000 subwords in the dict

In [ ]:
train_dev_input_shapes = (lambda x: [x['utts'][k]['input'][0]['shape'] for k in x['utts'].keys()])(train_dev_dict)
train_dev_output_shapes = (lambda x: [x['utts'][k]['output'][0]['shape'] for k in x['utts'].keys()])(train_dev_dict)
print(set(np.array(train_dev_input_shapes).T[1])) # should be correspondent to 80 fbanks 
print(set(np.array(train_dev_output_shapes).T[1])) # should be correspondent to 5000 subwords in the dict

In [ ]:
test_input_shapes = (lambda x: [x['utts'][k]['input'][0]['shape'] for k in x['utts'].keys()])(test_dict)
test_output_shapes = (lambda x: [x['utts'][k]['output'][0]['shape'] for k in x['utts'].keys()])(test_dict)
print(set(np.array(test_input_shapes).T[1])) # should be correspondent to 80 fbanks 
print(set(np.array(test_output_shapes).T[1])) # should be correspondent to 5000 subwords in the dict